### I. Implementieren Sie einen CollisionChecker, der es erlaubt, einen holonomen „mobilen“ Roboter mit 2 (x, y) bzw. 3 Freiheitsgraden (x, y, rot z) und Hindernissen mit den Planungsverfahren aus der Vorlesung zu planen, ohne dass die Planungsverfahren verändert werden müssen.

### a) Erläutern Sie, was „holonom“ in diesem Kontext bedeutet

Ein **holonomes System** ist ein System, bei dem die Anzahl der **steuerbaren Freiheitsgrade (Degrees of Freedom, DoF)** der Anzahl der **konfigurationsbeschreibenden Freiheitsgrade** entspricht.

Mit anderen Worten:  
Ein **Roboter ist holonom**, wenn er sich **jederzeit in alle Richtungen des Konfigurationsraums bewegen kann**, die durch seine mechanische Konfiguration erlaubt sind.

#### Im Kontext des Aufgabenblatts bedeutet das konkret:

- Ein **2-DoF mobiler Roboter** hat die Steuergrößen:
  - `x`: Translation entlang der x-Achse  
  - `y`: Translation entlang der y-Achse  

- Ein **3-DoF mobiler Roboter** hat die Steuergrößen:
  - `x`: Translation entlang der x-Achse  
  - `y`: Translation entlang der y-Achse  
  - `θ`: Rotation um die Hochachse (z-Achse)

Ein solcher Roboter kann sich also **unabhängig und direkt entlang jeder dieser Freiheitsgrade bewegen**, ohne Einschränkungen durch sogenannte **nicht-holonome Zwangsbedingungen** (wie etwa bei einem Auto, das nicht seitlich fahren kann).


In [ ]:

from shapely.geometry.base import BaseGeometry
from shapely import affinity
import random
import math

class SceneBuilder:
    """
    Baut eine Planungs-Szene mit beliebigen Hindernissen und Roboter-Basisgeometrien auf.
    """
    def __init__(self):
        self.obstacles: dict[str, BaseGeometry] = {}
        self.robots:    dict[str, BaseGeometry] = {}

    def add_obstacle(self, name: str, geom: BaseGeometry) -> None:
        self.obstacles[name] = geom

    def add_robot(self, name: str, base_geom: BaseGeometry) -> None:
        self.robots[name] = base_geom

    def get_obstacles(self) -> dict[str, BaseGeometry]:
        return self.obstacles

    def get_robot_base(self, name: str) -> BaseGeometry:
        return self.robots[name]


def transform_geometry(base: BaseGeometry, x: float, y: float, theta: float) -> BaseGeometry:
    deg = theta * 180.0 / math.pi
    rotated = affinity.rotate(base, deg, origin=(0, 0))
    return affinity.translate(rotated, xoff=x, yoff=y)


def generate_random_polygon_shape(num_vertices: int = 8,
                                  concavity: float = 0.3) -> BaseGeometry:
    angles = sorted(random.uniform(0, 2*math.pi) for _ in range(num_vertices))
    points = []
    for ang in angles:
        r = random.uniform(0.3, 1.0)
        if random.random() < concavity:
            r *= random.uniform(0.4, 0.8)
        points.append((r*math.cos(ang), r*math.sin(ang)))
    from shapely.geometry import Polygon, MultiPoint
    poly = Polygon(points)
    if not poly.is_valid or poly.area == 0:
        poly = MultiPoint(points).convex_hull
    return poly


def generate_letter_shape(letter: str, size: float = 1.0) -> BaseGeometry:
    from shapely.geometry import Polygon
    w = size * 0.2
    h = size
    if letter.upper() == 'I':
        shape = Polygon([(-w/2,0), (w/2,0), (w/2,h), (-w/2,h)])
    elif letter.upper() == 'T':
        top = Polygon([(-h/2,h), (h/2,h), (h/2,h-w), (-h/2,h-w)])
        stem = Polygon([(-w/2,0), (w/2,0), (w/2,h-w), (-w/2,h-w)])
        shape = top.union(stem)
    elif letter.upper() == 'K':
        vert = Polygon([(-w/2,0), (w/2,0), (w/2,h), (-w/2,h)])
        tri1 = Polygon([(w/2,h/2), (h/2,h), (h/2,h/2 + w)])
        tri2 = Polygon([(w/2,h/2), (h/2,0), (h/2,w)])
        shape = vert.union(tri1).union(tri2)
    else:
        return generate_random_polygon_shape()
    return shape


def generate_robot_shape(index: int) -> BaseGeometry:
    # Erzeuge Basisform
    if index == 0:
        base = generate_letter_shape('K', size=random.uniform(0.5,1.5))
    elif index == 1:
        base = generate_letter_shape('I', size=random.uniform(0.5,1.5))
    elif index == 2:
        base = generate_letter_shape('T', size=random.uniform(0.5,1.5))
    else:
        choice = random.random()
        if choice < 0.3:
            base = generate_random_polygon_shape(num_vertices=random.randint(5,8), concavity=0.2)
        elif choice < 0.6:
            base = generate_random_polygon_shape(num_vertices=random.randint(8,15), concavity=0.5)
        else:
            base = generate_letter_shape(random.choice(['K','I','T']), size=random.uniform(0.5,1.5))
    # Zufällige Skalierung
    scale = random.uniform(0.5, 2.0)
    return affinity.scale(base, xfact=scale, yfact=scale, origin=(0,0))

# Interaktives Notebook: 1–99 Roboter mit automatischer Skalierung
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    from shapely.geometry import Polygon, Point
    import ipywidgets as widgets
    from IPython.display import clear_output

    scene = SceneBuilder()
    scene.add_obstacle('wall', Polygon([(2,2),(18,2),(18,3),(2,3)]))
    scene.add_obstacle('pillar', Point(10,10).buffer(1.5))

    def plot_geom(ax, geom, **kwargs):
        if geom.geom_type == 'Polygon':
            xs, ys = geom.exterior.xy; ax.fill(xs, ys, **kwargs)
        elif geom.geom_type == 'MultiPolygon':
            for part in geom.geoms:
                xs, ys = part.exterior.xy; ax.fill(xs, ys, **kwargs)

    def update_scene(n_robots: int):
        clear_output(wait=True)
        scene.robots.clear()
        configs = []
        geoms = []
        for i in range(n_robots):
            shape = generate_robot_shape(i)
            name = f'robot_{i+1}'
            scene.add_robot(name, shape)
            x, y = random.uniform(0,20), random.uniform(0,20)
            theta = random.uniform(0,2*math.pi)
            configs.append((name,(x,y,theta)))
            geoms.append(transform_geometry(shape,x,y,theta))
        geoms += list(scene.get_obstacles().values())
        minx = min(g.bounds[0] for g in geoms)
        miny = min(g.bounds[1] for g in geoms)
        maxx = max(g.bounds[2] for g in geoms)
        maxy = max(g.bounds[3] for g in geoms)
        margin = max(maxx-minx,maxy-miny)*0.05

        fig, ax = plt.subplots(figsize=(6,6))
        for obs in scene.get_obstacles().values():
            plot_geom(ax, obs, fc='gray', ec='black', alpha=0.6)
        for name,(x,y,theta) in configs:
            geom = transform_geometry(scene.get_robot_base(name),x,y,theta)
            plot_geom(ax, geom, fc='none', ec='blue', lw=1.5)
        ax.set_aspect('equal')
        ax.set_xlim(minx-margin, maxx+margin)
        ax.set_ylim(miny-margin, maxy+margin)
        ax.set_title(f'Szene mit {n_robots} zufälligen Robotern')
        plt.show()

    slider = widgets.IntSlider(value=5, min=1, max=99, step=1, description='Anzahl:')
    widgets.interact(update_scene, n_robots=slider)


In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import box
from shapely.affinity import translate, rotate
from IPMobileRobotCollisionChecker import MobileRobotCollisionChecker

# Roboterform (zentriert um Ursprung)
robot_shape = box(-1, -0.5, 1, 0.5)

# Hindernisse
scene = {
    "obstacle1": box(5, 5, 8, 8),
    "obstacle2": box(10, 2, 12, 4)
}

# CollisionChecker
checker = MobileRobotCollisionChecker(robot_shape, scene, limits=[[0, 20], [0, 20], [0, 360]])

# Testpositionen
test_positions = [
    ([6, 5, 0], "Test 1"),
    ([4.6, 4, 45], "Test 2"),
    ([11, 8, 90], "Test 3")
]

# Visualisierung
fig, axs = plt.subplots(1, len(test_positions), figsize=(5 * len(test_positions), 5))

for ax, (pos, title) in zip(axs, test_positions):
    ax.set_title(f"{title} @ {pos}")
    ax.set_xlim(0, 20)
    ax.set_ylim(0, 20)
    ax.set_aspect("equal")

    # Hindernisse zeichnen
    for obs in scene.values():
        x, y = obs.exterior.xy
        ax.fill(x, y, color='red', alpha=0.5)

    # Roboterposition transformieren
    robot_transformed = rotate(robot_shape, pos[2], origin='centroid', use_radians=False)
    robot_transformed = translate(robot_transformed, xoff=pos[0], yoff=pos[1])

    # Kollision prüfen
    in_collision = checker.pointInCollision(pos)
    color = 'green' if not in_collision else 'orange'
    text_result = "Kollision" if in_collision else "Kollisionsfrei"

    # Roboter zeichnen
    x, y = robot_transformed.exterior.xy
    ax.fill(x, y, color=color, alpha=0.6)
    ax.plot(pos[0], pos[1], 'ko')

    # Ergebnistext anzeigen
    ax.text(0.5, -0.1, f"Checker: {text_result}", transform=ax.transAxes,
        fontsize=12, ha='center', va='top')


plt.tight_layout()
plt.show()


In [ ]:
from IPTestSuiteMR import benchList
import matplotlib.pyplot as plt
from shapely.affinity import rotate, translate

# Alle Benchmarks visualisieren
for benchmark in benchList:
    checker = benchmark.collisionChecker
    scene = checker.scene
    robot_shape = checker.robot_shape
    limits = checker.limits
    start = benchmark.startList[0]
    goal = benchmark.goalList[0]

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(*limits[0])
    ax.set_ylim(*limits[1])
    ax.set_aspect("equal")
    ax.set_title(f"{benchmark.name}: {benchmark.description}")

    # Hindernisse zeichnen
    for obs in scene.values():
        x, y = obs.exterior.xy
        ax.fill(x, y, color="red", alpha=0.5)

    # Roboter an Startposition
    if len(start) == 3:
        r_start = translate(rotate(robot_shape, start[2], origin="centroid"), xoff=start[0], yoff=start[1])
    else:
        r_start = translate(robot_shape, xoff=start[0], yoff=start[1])
    ax.fill(*r_start.exterior.xy, color="green", alpha=0.6)
    ax.text(start[0], start[1], "Start", ha="center", va="center")

    # Roboter an Zielposition
    if len(goal) == 3:
        r_goal = translate(rotate(robot_shape, goal[2], origin="centroid"), xoff=goal[0], yoff=goal[1])
    else:
        r_goal = translate(robot_shape, xoff=goal[0], yoff=goal[1])
    ax.fill(*r_goal.exterior.xy, color="blue", alpha=0.4)
    ax.text(goal[0], goal[1], "Ziel", ha="center", va="center")

    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import matplotlib as mpl
from shapely.affinity import translate, rotate
import numpy as np
from IPython.display import HTML, display
from IPBasicPRM import BasicPRM
from IPLazyPRM import LazyPRM
from tqdm.notebook import tqdm

# Globale Einstellungen
mpl.rcParams['animation.embed_limit'] = 100

# ---------------------------------
# PRM-Konfigurationen
# ---------------------------------


# Maximale Anzahl von Versuchen pro Benchmark (BasicPRM)
max_attempts = 100

# -----------------------
# Methoden-Definitionen
# -----------------------

def transform_robot(pos, robot_shape):
    x, y = pos[0], pos[1]
    theta = pos[2] if len(pos) == 3 else 0
    r = rotate(robot_shape, theta, origin='centroid', use_radians=False)
    return translate(r, xoff=x, yoff=y)


def interpolate_path(path, steps_per_segment=10):
    interp_path = []
    for i in range(len(path) - 1):
        p1 = np.array(path[i])
        p2 = np.array(path[i + 1])
        for j in range(steps_per_segment):
            alpha = j / steps_per_segment
            interp_path.append((1 - alpha) * p1 + alpha * p2)
    interp_path.append(np.array(path[-1]))
    return interp_path


def compute_prm_path(planner_cls, collision_checker, start, goal, config):
    planner = planner_cls(collision_checker)
    try:
        path_ids = planner.planPath([start], [goal], config)
        return path_ids, planner.graph
    except Exception as e:
        print(f"❌ Fehler bei Pfadplanung mit {planner_cls.__name__}: {e}")
        return [], planner.graph



def plot_configuration_space(ax, graph, path, start, goal, dof, collision_checker):
    if dof == 3:
        for u, v in graph.edges():
            p1, p2 = graph.nodes[u]['pos'], graph.nodes[v]['pos']
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='gray', linewidth=0.3)
        pts = np.array([graph.nodes[n]['pos'] for n in graph.nodes()])
        ax.scatter(pts[:,0], pts[:,1], pts[:,2], c='k', s=2)
        path_xyz = np.array(path)
        ax.plot(path_xyz[:,0], path_xyz[:,1], path_xyz[:,2], 'b-', linewidth=2)
        ax.scatter(start[0], start[1], start[2], c='g', s=50, marker='o')
        ax.scatter(goal[0], goal[1], goal[2], c='r', s=50, marker='^')
        robot_dot = ax.scatter([], [], [], c='r', s=30)
    else:
        ax.set_xlim(collision_checker.limits[0])
        ax.set_ylim(collision_checker.limits[1])
        for u, v in graph.edges():
            p1, p2 = graph.nodes[u]['pos'], graph.nodes[v]['pos']
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='gray', linewidth=0.3)
        pts = np.array([graph.nodes[n]['pos'] for n in graph.nodes()])
        ax.plot(pts[:,0], pts[:,1], 'k.', markersize=2)
        pa = np.array(path)
        ax.plot(pa[:,0], pa[:,1], 'b-', linewidth=2)
        ax.scatter(start[0], start[1], c='g', s=50)
        ax.scatter(goal[0], goal[1], c='r', s=50)
        robot_dot, = ax.plot([], [], 'ro', markersize=6)
    return robot_dot


def plot_work_space(ax, scene, robot_shape, start, goal, collision_checker):
    limits = collision_checker.limits
    ax.set_xlim(limits[0]); ax.set_ylim(limits[1])
    for obs in scene.values():
        x, y = obs.exterior.xy
        ax.fill(x, y, color='red', alpha=0.5)
    start_poly = transform_robot(start, robot_shape)
    goal_poly  = transform_robot(goal,  robot_shape)
    ax.fill(*start_poly.exterior.xy, color='green', alpha=0.4)
    ax.fill(*goal_poly.exterior.xy,  color='blue',  alpha=0.4)
    patch = plt.Polygon(np.array(start_poly.exterior.coords), closed=True, fc='orange', alpha=0.8)
    ax.add_patch(patch)
    return patch


def make_animation(fig, init_func, animate_func, n_frames, interval=100):
    return animation.FuncAnimation(
        fig, animate_func, frames=n_frames,
        init_func=init_func, blit=True, interval=interval, repeat=False
    )

In [ ]:

# ---------------------------------
# BasicPRM-Visualisierung mit Retries
# ---------------------------------
config_basic = {'radius': 12.0, 'numNodes': 900}
bench_range_basic   = benchList[0:5]  # Passe die Anzahl nach Bedarf an
for idx, benchmark in enumerate(bench_range_basic):
    start = benchmark.startList[0]
    goal  = benchmark.goalList[0]
    dof   = len(start)
    path_ids = []

    
    for attempt in tqdm(range(1, max_attempts + 1), desc=f"BasicPRM Benchmark {idx+1}/{len(bench_range_basic)}"):  
        path_ids, graph = compute_prm_path(BasicPRM, benchmark.collisionChecker, start, goal, config_basic)
        if path_ids:
            print(f"✔️ BasicPRM Benchmark {idx}: Pfad gefunden nach {attempt} Versuchen.")
            break
    if not path_ids:
        print(f"❌ BasicPRM Benchmark {idx}: Kein Pfad nach {max_attempts} Versuchen.")
        continue

    # Visualisierung
    path = [graph.nodes[n]['pos'] for n in path_ids]
    interp = interpolate_path(path)
    fig = plt.figure(figsize=(14,7))
    ax1 = fig.add_subplot(1,2,1, projection='3d') if dof==3 else fig.add_subplot(1,2,1)
    ax1.set_title('Konfigurationsraum (BasicPRM)')
    if dof == 3:
        coord1 = ax1.text2D(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                             verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    else:
        coord1 = ax1.text(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                          verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title('Arbeitsraum (BasicPRM)')
    coord2 = ax2.text(0.02, 0.95, '', transform=ax2.transAxes, fontsize=12,
                      verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    robot_dot = plot_configuration_space(ax1, graph, path, start, goal, dof, benchmark.collisionChecker)
    robot_patch = plot_work_space(ax2, benchmark.collisionChecker.scene,
                                   benchmark.collisionChecker.robot_shape,
                                   start, goal, benchmark.collisionChecker)
    def init():
        robot_patch.set_xy(np.array(transform_robot(start, benchmark.collisionChecker.robot_shape).exterior.coords))
        if dof!=3:
            robot_dot.set_data([], [])
        else:
            robot_dot._offsets3d=([],[],[])
        coord1.set_text('')
        coord2.set_text('')
        return robot_patch, robot_dot, coord1, coord2
    def animate(i):
        pos=interp[min(i,len(interp)-1)]
        transformed=transform_robot(pos, benchmark.collisionChecker.robot_shape)
        robot_patch.set_xy(np.array(transformed.exterior.coords))
        if dof==3:
            robot_dot._offsets3d=([pos[0]],[pos[1]],[pos[2]])
        else:
            robot_dot.set_data([pos[0]],[pos[1]])
        txt=f"x={pos[0]:.2f}, y={pos[1]:.2f}"+(f", θ={pos[2]:.2f}" if dof==3 else "")
        coord1.set_text(txt)
        coord2.set_text(txt)
        return robot_patch,robot_dot,coord1,coord2
    ani=make_animation(fig,init,animate,len(interp))
    display(HTML(ani.to_jshtml()))
    plt.close(fig)

In [ ]:
# ---------------------------------
# LazyPRM-Visualisierung mit Retries
# ---------------------------------

# Benchmarks für LazyPRM
bench_range_lazy = benchList[0:5]  # Beispiel: andere Benchmarks oder gleiche Range


config_lazy = {'initialRoadmapSize': 200, 'updateRoadmapSize': 50,
               'kNearest': 10, 'maxIterations': 50}

for idx, benchmark in enumerate(bench_range_lazy):
    start = benchmark.startList[0]
    goal  = benchmark.goalList[0]
    dof   = len(start)
    path_ids = []

    for attempt in tqdm(range(1, max_attempts + 1), desc=f"LazyPRM Benchmark {idx+1}/{len(bench_range_lazy)}"):
        path_ids, graph = compute_prm_path(LazyPRM, benchmark.collisionChecker, start, goal, config_lazy)
        if path_ids:
            print(f"✔️ LazyPRM Benchmark {idx}: Pfad gefunden nach {attempt} Versuchen.")
            break
    if not path_ids:
        print(f"❌ LazyPRM Benchmark {idx}: Kein Pfad nach {max_attempts} Versuchen.")
        continue

    path = [graph.nodes[n]['pos'] for n in path_ids]
    interp = interpolate_path(path)
    fig = plt.figure(figsize=(14,7))
    ax1 = fig.add_subplot(1,2,1, projection='3d') if dof==3 else fig.add_subplot(1,2,1)
    ax1.set_title('Konfigurationsraum (LazyPRM)')
    if dof == 3:
        coord1 = ax1.text2D(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                             verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    else:
        coord1 = ax1.text(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                          verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title('Arbeitsraum (LazyPRM)')
    coord2 = ax2.text(0.02, 0.95, '', transform=ax2.transAxes, fontsize=12,
                      verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    robot_dot = plot_configuration_space(ax1, graph, path, start, goal, dof, benchmark.collisionChecker)
    robot_patch = plot_work_space(ax2, benchmark.collisionChecker.scene,
                                   benchmark.collisionChecker.robot_shape,
                                   start, goal, benchmark.collisionChecker)

    def init():
        robot_patch.set_xy(np.array(transform_robot(start, benchmark.collisionChecker.robot_shape).exterior.coords))
        if dof!=3:
            robot_dot.set_data([], [])
        else:
            robot_dot._offsets3d=([],[],[])
        coord1.set_text('')
        coord2.set_text('')
        return robot_patch, robot_dot, coord1, coord2

    def animate(i):
        pos = interp[min(i, len(interp)-1)]
        transformed = transform_robot(pos, benchmark.collisionChecker.robot_shape)
        robot_patch.set_xy(np.array(transformed.exterior.coords))
        if dof == 3:
            robot_dot._offsets3d = ([pos[0]], [pos[1]], [pos[2]])
        else:
            robot_dot.set_data([pos[0]], [pos[1]])
        txt = f"x={pos[0]:.2f}, y={pos[1]:.2f}" + (f", θ={pos[2]:.2f}" if dof == 3 else "")
        coord1.set_text(txt)
        coord2.set_text(txt)
        return robot_patch, robot_dot, coord1, coord2

    ani = make_animation(fig, init, animate, len(interp))
    display(HTML(ani.to_jshtml()))
    plt.close(fig)


In [ ]:
from IPRRT import RRTSimple
# ---------------------------------
# RRTSimple-Visualisierung mit Retries
# ---------------------------------

bench_range_rrt = benchList[0:5]  # Beispielbereich anpassen

config_rrt = {'numberOfGeneratedNodes': 500, 'testGoalAfterNumberOfNodes': 10}

for idx, benchmark in enumerate(bench_range_rrt):
    start = benchmark.startList[0]
    goal  = benchmark.goalList[0]
    dof   = len(start)
    path_ids = []

    for attempt in tqdm(range(1, max_attempts + 1), desc=f"RRT Benchmark {idx+1}/{len(bench_range_rrt)}"):
        path_ids, graph = compute_prm_path(RRTSimple, benchmark.collisionChecker, start, goal, config_rrt)
        if path_ids:
            print(f"✔️ RRT Benchmark {idx}: Pfad gefunden nach {attempt} Versuchen.")
            break
    if not path_ids:
        print(f"❌ RRT Benchmark {idx}: Kein Pfad oder Fehler nach {max_attempts} Versuchen.")
        continue

    path = [graph.nodes[n]['pos'] for n in path_ids]
    interp = interpolate_path(path)
    fig = plt.figure(figsize=(14,7))
    ax1 = fig.add_subplot(1,2,1, projection='3d') if dof==3 else fig.add_subplot(1,2,1)
    ax1.set_title('Konfigurationsraum (RRT)')
    if dof == 3:
        coord1 = ax1.text2D(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                             verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    else:
        coord1 = ax1.text(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12,
                          verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title('Arbeitsraum (RRT)')
    coord2 = ax2.text(0.02, 0.95, '', transform=ax2.transAxes, fontsize=12,
                      verticalalignment='top', bbox=dict(facecolor='white',alpha=0.7))
    robot_dot = plot_configuration_space(ax1, graph, path, start, goal, dof, benchmark.collisionChecker)
    robot_patch = plot_work_space(ax2, benchmark.collisionChecker.scene,
                                   benchmark.collisionChecker.robot_shape,
                                   start, goal, benchmark.collisionChecker)

    def init():
        robot_patch.set_xy(np.array(transform_robot(start, benchmark.collisionChecker.robot_shape).exterior.coords))
        if dof!=3:
            robot_dot.set_data([], [])
        else:
            robot_dot._offsets3d=([],[],[])
        coord1.set_text('')
        coord2.set_text('')
        return robot_patch, robot_dot, coord1, coord2

    def animate(i):
        pos = interp[min(i, len(interp)-1)]
        transformed = transform_robot(pos, benchmark.collisionChecker.robot_shape)
        robot_patch.set_xy(np.array(transformed.exterior.coords))
        if dof == 3:
            robot_dot._offsets3d = ([pos[0]], [pos[1]], [pos[2]])
        else:
            robot_dot.set_data([pos[0]], [pos[1]])
        txt = f"x={pos[0]:.2f}, y={pos[1]:.2f}" + (f", θ={pos[2]:.2f}" if dof == 3 else "")
        coord1.set_text(txt)
        coord2.set_text(txt)
        return robot_patch, robot_dot, coord1, coord2

    ani = make_animation(fig, init, animate, len(interp))
    display(HTML(ani.to_jshtml()))
    plt.close(fig)